<a href="https://colab.research.google.com/github/Paulina9555/Test/blob/master/KAGGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -qq datasets
! pip install -qq transformers==4.28.0
! pip -qq install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 341.0 kB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import Dataset
import numpy as np
import torch
from transformers import AutoTokenizer,DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
print(df.shape)

(7613, 5)


In [ ]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
df = df[df['text'].str.len() > 10]
df.shape

(7602, 5)

In [ ]:
df['label'] = df['target'].astype(int)
df['text'] = df['text'].str.lower()
dataset = Dataset.from_pandas(df[['text','label']])

<ipython-input-5-42663ec684d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['target'].astype(int)
<ipython-input-5-42663ec684d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()


In [ ]:
df['label'].nunique()

2

In [ ]:
dataset = dataset.train_test_split(test_size = 0.3)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length = 512)

In [ ]:
dataset['train'][0]

{'text': "moved on to 'bang bang rock and roll' by @art_brut_ . it's been too long since i've played this one loud. art brut top of the pops.",
 'label': 0,
 '__index_level_0__': 4806}

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5321 [00:00<?, ? examples/s]

Map:   0%|          | 0/2281 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import datasets
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    metric1 = datasets.load_metric("accuracy")
    metric2 = datasets.load_metric("f1")


    predictions = np.argmax(logits, axis=-1)
    accuracy = round(metric1.compute(predictions=predictions, references=labels)['accuracy'], 3)
    f1 = round(metric2.compute(predictions=predictions, references=labels, average="macro")['f1'],3 )
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large", num_labels=2, ignore_mismatched_sizes=True)

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [ ]:
training_args = TrainingArguments(
    output_dir="models",
    learning_rate=2e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    group_by_length=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.463667,0.813000,0.809000
2,0.486400,0.405211,0.836000,0.829000
3,0.486400,0.471923,0.833000,0.827000
4,0.354900,0.461646,0.838000,0.831000
5,0.299700,0.466646,0.844000,0.834000
6,0.299700,0.561301,0.834000,0.828000
7,0.272200,0.541291,0.838000,0.831000
8,0.247800,0.587665,0.837000,0.830000
9,0.247800,0.630309,0.835000,0.828000
10,0.222500,0.631027,0.836000,0.829000


<ipython-input-13-f5f7f56f3815>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric1 = datasets.load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect th

TrainOutput(global_step=3330, training_loss=0.30454267438825544, metrics={'train_runtime': 2705.7233, 'train_samples_per_second': 19.666, 'train_steps_per_second': 1.231, 'total_flos': 3179627483055180.0, 'train_loss': 0.30454267438825544, 'epoch': 10.0})

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained("/content/models/checkpoint-3330", num_labels=2, ignore_mismatched_sizes=True).to("cuda")

In [ ]:
from tqdm import tqdm

In [ ]:
test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [ ]:
sentences = list(test['text'])

In [ ]:
batch_size = 8
batches = [sentences[i:i + batch_size] for i in range(0, len(sentences), batch_size)]

In [ ]:
pred = []
for t in tqdm(batches):
    encoded_inputs = tokenizer(t, padding=True, truncation=True, return_tensors='pt').to("cuda")
    outputs = model(**encoded_inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    predicted_labels = torch.argmax(probs, dim=1).tolist()
    pred.append(predicted_labels)

100%|██████████| 408/408 [00:47<00:00,  8.65it/s]


In [ ]:
sum([['a','b'],['a','b']], [] )

['a', 'b', 'a', 'b']

In [ ]:
pred = sum(pred, [])

In [ ]:
test["target"] = pred

In [ ]:
test[['id','target']].to_csv("my_sub.csv", index = False)

In [ ]:
test[['id','target']]

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
